In [2]:

import os, shutil
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import cv2
import shutil

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
#get working directory
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project\\fatpad'

In [5]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'brucechou1983_CheXNet_Keras_0.3.0_weights.h5',
 'CNN_automated_medical_diagnostics_five_layers_chopped_two_classes_trained_on_twenty_epochs_Adam_0.0001.ipynb',
 'CNN_FATPAD_SimplePlainNN.ipynb',
 'CNN_FATPAD_TransferLearning-DenseNet.ipynb',
 'CNN_FATPAD_TransferLearning-EfficientNet.ipynb',
 'CNN_FATPAD_TransferLearning-NASNet.ipynb',
 'CNN_FATPAD_TransferLearning-ResNet.ipynb',
 'CNN_FATPAD_TransferLearning-Xception.ipynb',
 'CNN_FATPAD_TransferLearning.ipynb',
 'Labels_FatPadDetection.xlsx',
 'Prepare_Xray_Data.ipynb',
 'README.md',
 '~$Labels_FatPadDetection.xlsx']

In [7]:
#change working directory to where the datasets are
root_directory=os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project')

In [8]:
#Provide train, validation, and test to tensorflow-keras ImageDataGenerator
base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train'
base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/validation'
base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/test'

print(base_dir_train)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train


In [37]:
#run tensorflow-keras ImageDataGenerator on the directory which contains train, validate and test
#train_datagen = ImageDataGenerator(
 #   rescale=1./255, # new parameter (maps pixel values to floats)
 #   rotation_range=40,
 #   width_shift_range=0.2,
  ##  height_shift_range=0.2,
  #  shear_range=0.2,
  #  zoom_range=0.25,
  #  horizontal_flip=True,
  #  fill_mode='nearest') # Note: I added this, too

train_datagen = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              rotation_range=5, 
                              fill_mode = 'reflect',
                              zoom_range=0.15)

# The validation data should not be augmented (just rescaled)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_dir_train,
        color_mode='rgb',
        target_size=(224, 224), 
        batch_size=16, 
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        base_dir_validate, 
        color_mode='rgb',
        target_size=(224, 224), 
        batch_size=16, 
        class_mode='binary')

Found 3519 images belonging to 2 classes.
Found 1005 images belonging to 2 classes.


In [49]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

def plot_history(h):
    acc = h.history['acc']
    val_acc = h.history['val_acc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [40]:
train_generator.image_shape
from matplotlib import pyplot
gr=0
if gr: 
    fig, axs = plt.subplots(4,4, figsize=(15, 15), facecolor='w', edgecolor='k')
    axs = axs.ravel()
    for i in range(16):
        # define subplot
        # generate batch of images
        batch = train_generator.next()
        # convert to unsigned integers for viewing
        image = batch[0]
        # plot raw pixel data
        axs[i].imshow(image[i].reshape(224,224), cmap=pyplot.get_cmap('gray'))# show the figure

In [72]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
inputs = layers.Input(shape=train_generator.image_shape)

base_model = keras.applications.xception.Xception(input_tensor=inputs,weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(1, activation="sigmoid")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(lr=0.3, momentum=0.9, decay=0.01)
#optimizer = keras.optimizers.Adam(learning_rate=1e-2)

model.compile(loss="binary_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________

In [73]:
checkpoint_cb = tensorflow.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tensorflow.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

In [74]:
initial_learning_rate = 0.015
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

In [75]:
METRICS = [
    tensorflow.keras.metrics.BinaryAccuracy(),
    tensorflow.keras.metrics.Precision(name="precision"),
    tensorflow.keras.metrics.Recall(name="recall"),
]


In [76]:
batch_size=16
hist = model.fit(train_generator,
      epochs=100,
    steps_per_epoch=int(0.75 * train_generator.n / batch_size),
      validation_data=validation_generator,
     validation_steps=int(0.15 * validation_generator.n / batch_size),
      verbose=1,callbacks=[checkpoint_cb, early_stopping_cb])




Epoch 1/100
164/164 [==============================] - 45s 275ms/step - loss: 5.6156 - accuracy: 0.5627 - val_loss: 6.2194 - val_accuracy: 0.5278
Epoch 2/100
164/164 [==============================] - 43s 265ms/step - loss: 3.0224 - accuracy: 0.5902 - val_loss: 3.4246 - val_accuracy: 0.5417
Epoch 3/100
164/164 [==============================] - 43s 264ms/step - loss: 2.0713 - accuracy: 0.6052 - val_loss: 2.4621 - val_accuracy: 0.5417
Epoch 4/100
164/164 [==============================] - 43s 263ms/step - loss: 1.4990 - accuracy: 0.6077 - val_loss: 1.6577 - val_accuracy: 0.5278
Epoch 5/100
164/164 [==============================] - 43s 264ms/step - loss: 1.3301 - accuracy: 0.6233 - val_loss: 1.6454 - val_accuracy: 0.5208
Epoch 6/100
164/164 [==============================] - 43s 265ms/step - loss: 1.0809 - accuracy: 0.6428 - val_loss: 0.8467 - val_accuracy: 0.6597
Epoch 7/100
164/164 [==============================] - 43s 263ms/step - loss: 0.9489 - accuracy: 0.6496 - val_loss: 1.2725 -

In [51]:
plot_hist(hist)

KeyError: 'acc'

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=20)
test_acc